In [2]:
# imports
import torch 
import torchvision
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F # All functions that don't have any parameters
import torchvision.transforms as transforms # Transformations we can perform on our dataset
import torchvision.datasets as datasets # Standard datasets
from torch.utils.data import DataLoader # Gives easier dataset management and creates mini batches
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler







In [6]:

# This mps is for Macos M1 chip. If you are using a different device, you can use "cuda" for GPU or "cpu" for CPU
device = torch.device("mps")

In [15]:
# Hyperparameters
input_size = 28
hidden_size = 256
num_layers = 2
num_classes = 10
sequence_length = 28
learning_rate = 0.005
batch_size = 64
num_epochs = 3

In [16]:
# load data
train_dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root="dataset/", train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


In [26]:
# create a bidirectional LSTM
class biLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(biLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.biLstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)         
        self.fc = nn.Linear(hidden_size* 2, num_classes)


    def forward(self, x):
        """
        x: shape of x is (batch_size, sequence_length, input_size)
        In this method, we will implement the forward pass of the model
        h0 and c0 are initial hidden and cell states for the LSTM 
        They are initialized to 0

        * h0: initial hidden state for each element in the batch. Its shape is (num_layers * num_directions, batch_size, hidden_size)
        Because this is a bidirectional LSTM, num_directions is 2

        c0 is the initial cell state for each element in the batch. Its shape is (num_layers * num_directions, batch_size, hidden_size)

        """
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device) 
        

        out, (hidden_state, cell_state) = self.biLstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out





In [27]:
# Initialize network
Model = biLSTM(input_size, hidden_size, num_layers, num_classes).to(device)

In [28]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Model.parameters(), lr=learning_rate)


In [29]:


# train network 
for epochs in range(num_epochs):   
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # forward
        scores = Model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
        

In [31]:
# check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")
    model.train()

check_accuracy(train_loader, Model)
check_accuracy(test_loader, Model)

Checking accuracy on training data
Got 59066 / 60000 with accuracy 98.44
Checking accuracy on test data
Got 9794 / 10000 with accuracy 97.94
